# EPA IDS Data
To supplement ECHO’s record of formal regulatory actions, we included EPA’s IDS data to provide a secondary lens on pesticide exposure risks. IDS incident trends offer insight into underreported regions, populations at risk, or failure points in state-level WPS enforcement systems.
## Data Overview 

This dataset includes self-reported pesticide-related incidents from 2014-2024. It captures the type of incident (Human-related is the one I focused on but theres ~5 different types), the state it occured  in, and the county if available. This data offers an alternative to the echo data in how incidents within the agriculture space   are treated. 

## Data Collection
Data was obtained from scraping from the EPA IDS website
Queries were left blank to allow for every possible incident to be analyzed 

## Data Structure 
Incident Data (DateTime Object): When the incident was reported 
Reason for Report (String): 2 word summary of incident  
Impact of Incident (String): Denotes the incident to a 2-letter code, ie HC for Human Moderate 
Country (String): Country of Occurrence  
State (String): State of Occurrence 
County (String): County of  Occurrence 
City(String): City of Occurrence 
Product Reg # (String): If a pesticide was involved or reported what is the # code 
Product Name (String): If a pesticide was involved what is the product 
PC Code (Int): An EPA designated code for each pesticide 
Active Ingredients (String): The actice ingredient in the pesticide 
Clean (String): A cleaned version of what happened with the incident  

In [1]:
import pandas as pd

# Load raw data
df = pd.read_csv("IDS.csv")

# show data size and the first 5 raws
print("dataset size: ", len(df))
df.head()


dataset size:  18609


,Incident Number,Incident Date,Reason for Report,Impact of Incident,Country,State,County,City,Product Registration Number(s),Product Names,PC Codes,Active Ingredient(s),Overall Submission Description (may describe multiple incidents)
0,026567-00010,02/26/2014,Adverse Reaction,ON - Other Nontarget,US,CA,Merced,NaN,NaN,NaN,035001,Dimethoate,"US EPA Region 9: 21 reports including updates,..."
1,026567-00018,05/08/2014,Adverse Reaction,ON - Other Nontarget,US,CA,Tulare,NaN,NaN,NaN,NaN,Unknown Ingredients,"US EPA Region 9: 21 reports including updates,..."
2,026800-00005,05/14/2014,Adverse Reaction,ON - Other Nontarget,CN,NaN,Ontario,Grey,NaN,NaN,044309,Clothianidin,"8 Canadian bee kill incidents, Ontario, 2012, ..."
3,026800-00005,05/14/2014,Adverse Reaction,ON - Other Nontarget,CN,NaN,Ontario,Grey,NaN,NaN,109302,Fluvalinate,"8 Canadian bee kill incidents, Ontario, 2012, ..."
4,026800-00005,05/14/2014,Adverse Reaction,ON - Other Nontarget,CN,NaN,Ontario,Grey,NaN,NaN,099050,Acetamiprid,"8 Canadian bee kill incidents, Ontario, 2012, ..."


## Data Processing 
Afterwards data was ran through topic modeling and an LLM to help cut down the data to only be agricultural related 

Then the cut down data was imported into python and further cleaned through Regex, only including american incidents, dropping unncessary coluns, and making sure to only include incidents that are related to humans 

Data Quality & Limitations 

The biggest issue with the data is the fact that it is self-reported. This means that a lot of the incident descriptions are not very well detailed and it also leaves to the possibility that many incidents are not reported. To combat this I plan to pair the data with the ECHO data to use the IDS as a way to see which states are most compliant with policies.  

In [2]:
# remove column names and empty space
df.columns = df.columns.str.strip()
df["Country"] = df["Country"].str.strip().str.upper()

# filter for human-related incidents
df_human = df[df["Impact of Incident"].str.contains("H", na=False)]

# filter for U.S.-based incidents
df_human = df_human[df_human["Country"].str.upper() == "US"]

# Filtered dataset size
print("Filtered dataset size: ", len(df_human))
df_human.head()

Filtered dataset size:  13011


,Incident Number,Incident Date,Reason for Report,Impact of Incident,Country,State,County,City,Product Registration Number(s),Product Names,PC Codes,Active Ingredient(s),Overall Submission Description (may describe multiple incidents)
5,027212-00004,11/07/2014,Adverse Reaction,"HB - Human - Major, HC - Human - Moderate",US,MS,NaN,NaN,000100-00990,Demon Wp Insecticide,109702,Cypermethrin,"Syngenta: 1 H-B and 4 H-C individual reports, ..."
6,027220-00001,11/06/2014,Product Defect,HC - Human - Moderate,US,VA,Augusta,Weyers Cave,072959-00006,Degesch Fumi Cel,066504,Magnesium phosphide,Degesch America: 1 H-C: trauma & severe bruisi...
7,027251-00001,11/17/2014,Adverse Reaction,HA - Human Fatality,US,CO,NaN,Briscoe,005185-00505-080306,The Works Toilet Bowl Cleaner,045901,Hydrochloric acid,Biolab: 1 H-A: male's suicidal death involving...
8,027251-00001,11/17/2014,Adverse Reaction,HA - Human Fatality,US,CO,NaN,Briscoe,NaN,Hi Yield Lime Sulfur Spray,076702,Lime sulfur,Biolab: 1 H-A: male's suicidal death involving...
9,027261-00001,11/23/2014,Adverse Reaction,"HC - Human - Moderate, HD - Human - Minor",US,KY,NaN,Science Hill,NaN,Total Pest Control,109701,Permethrin,NPIC: woman and her daughter experienced sympt...


In [3]:
import re

def remove_years(text):
    return re.sub(r'\b(20[0-2][0-9]|19[8-9][0-9])\b', '', text)

BRANDS = ["monsanto", "basf", "bayer", "jk", "sc", "johnson"]
COMMON_NOISE = ["report", "reports", "update", "summaries", "entered", "annual", "aggregate"]

def remove_noise_words(text):
    text = text.lower()
    for word in BRANDS + COMMON_NOISE:
        text = text.replace(word, '')
    return text

In [4]:
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
tokenizer = TreebankWordTokenizer()

def lemmatize_text(text):
    words = tokenizer.tokenize(text)
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def clean_for_lda(text):
    text = str(text).lower()
    text = remove_years(text)
    text = remove_noise_words(text)
    text = lemmatize_text(text)
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    return text


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Use 'clean' if available, else use fallback field
text_col = "Clean" if "Clean" in df.columns else "Overall Submission Description (may describe multiple incidents)"

# Fill missing values with empty strings
texts_cleaned = df[text_col].fillna("").apply(clean_for_lda)

# Convert text into vectorized form (bag of words)
vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=10)
X = vectorizer.fit_transform(texts_cleaned)

# Fit and LDA model with 5 topics
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

# Print top words in each topic
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        top_words = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-top_n:]]
        print(f"Topic {idx}: {top_words}")

print_topics(lda, vectorizer)

Topic 0: ['near', 'incident', 'locate', 'pesticide', 'station', 'university', 'field', 'kill', 'efed', 'bee']
Topic 1: ['pruritus', 'hd', 'pain', 'unite', 'pa', 'irritation', 'hb', 'symptoms', 'hc', 'include']
Topic 2: ['incident', 'expose', 'ed', 'application', 'npic', 'oct', 'pesticide', 'experience', 'symptoms', 'region']
Topic 3: ['benckiser', '514', '746', '1294', 'pb', 'engenia', 'efficacy', 'lack', 'ids', 'incidents']
Topic 4: ['season', 'rainwater', 'stream', 'hb', 'grow', 'aug', 'july', 'hc', 'individual', 'incident']


In [6]:
# Get topic distribution per incident
topic_distribution = lda.transform(X)

# Assign the most probable topic to each record
df['Topic'] = topic_distribution.argmax(axis=1)

# Show how many records per topic
df['Topic'].value_counts()

Topic
1    7772
2    3004
4    2772
3    2726
0    2335
Name: count, dtype: int64

In [7]:
# Set agriculture-related topic based on keyword inspection
agri_topic_id = 2

# Filter incidents predicted to belong to this topic
df_agriculture = df[df['Topic'] == agri_topic_id]

# Save filtered results
df_agriculture.to_csv("IDS_agriculture.csv", index=False)

print(f"Saved {len(df_agriculture)} agriculture-related records.")
df_agriculture.head()

Saved 3004 agriculture-related records.


,Incident Number,Incident Date,Reason for Report,Impact of Incident,Country,State,County,City,Product Registration Number(s),Product Names,PC Codes,Active Ingredient(s),Overall Submission Description (may describe multiple incidents),Topic
0,026567-00010,02/26/2014,Adverse Reaction,ON - Other Nontarget,US,CA,Merced,NaN,NaN,NaN,035001,Dimethoate,"US EPA Region 9: 21 reports including updates,...",2
1,026567-00018,05/08/2014,Adverse Reaction,ON - Other Nontarget,US,CA,Tulare,NaN,NaN,NaN,NaN,Unknown Ingredients,"US EPA Region 9: 21 reports including updates,...",2
6,027220-00001,11/06/2014,Product Defect,HC - Human - Moderate,US,VA,Augusta,Weyers Cave,072959-00006,Degesch Fumi Cel,066504,Magnesium phosphide,Degesch America: 1 H-C: trauma & severe bruisi...,2
7,027251-00001,11/17/2014,Adverse Reaction,HA - Human Fatality,US,CO,NaN,Briscoe,005185-00505-080306,The Works Toilet Bowl Cleaner,045901,Hydrochloric acid,Biolab: 1 H-A: male's suicidal death involving...,2
8,027251-00001,11/17/2014,Adverse Reaction,HA - Human Fatality,US,CO,NaN,Briscoe,NaN,Hi Yield Lime Sulfur Spray,076702,Lime sulfur,Biolab: 1 H-A: male's suicidal death involving...,2


After training the LDA model, I reviewed each topic’s top keywords. Topic 0 included highly agriculture-specific terms such as 'glyphosate', 'dicamba', 'monsanto', and 'growing', indicating it represents farm-related pesticide usage. I then filtered the dataset to include only incidents classified under this topic.